In [ ]:
import pandas as pd

data=pd.read_csv('data/HousePrice/上海楼盘.csv')
data

In [1]:
# @desc: 利用高德API下载地理编码
 
import requests
import pandas as pd
import os
import time
import transbigdata as tbd
from time import sleep
import random
 
# 导入excel
# amp_api_data = pd.read_excel(r'input.xlsx')
# amp_api_data = pd.read_csv(r'data/fauck.csv',encoding='gbk')
# amp_api_data = pd.read_csv(r'out/poinameaddress.csv')
# amp_api_data=pd.read_csv('data/HousePrice/上海楼盘.csv')
amp_api_data=pd.read_csv('data/HousePrice/上海楼盘temp.csv')
# amp_api_data.columns=['标题','图片','地址_链接','listmap','huxing','顶','huxing1','huxing2','huxing3','wxqrtext','状态','状态4','标签','标签5','标签6','标签7','标签8','图片9','标记_链接','标记','价格','价格10']


# 没有上海市的地址在前面加上上海市 将POI搜索范围锁定
if 'listmap' in amp_api_data and '上海市' not in amp_api_data.get('listmap', ''):
    amp_api_data['listmap'] = '上海市' + amp_api_data.get('listmap', '')
amp_api_data

#  请求地址前缀
req_url_pref = "https://restapi.amap.com/v3/geocode/geo?"
amp_api_key = '61c256acd1b853e34a6b9a31033c399d'
result = pd.DataFrame()  # 初始化
count = 0
print("---------------------------------------------------------------")
def get_poi_from_amap():
    """
    从高德地图API下载地理编码
    """
    global result
    global count
    response = requests.get(req_url_pref, params=rep_params)
    data = response.json()  # 返回字典数据dict
    status = data["status"]
    if status != "0":
        address = data["geocodes"][0]["formatted_address"]
        lng = float(data["geocodes"][0]["location"].split(",")[0])
        lat = float(data["geocodes"][0]["location"].split(",")[1])
        wgs84_lng,wgs84_lat=tbd.gcj02towgs84(lng,lat)
        busi_data = [
            {
                "input_name": amp_api_name,
                "price":price,
                "aveprice":aveprice,
                "input_address": amp_api_address,
                "output_address": address,
                "gcj02_lng": lng,
                "gcj02_lat": lat,
                "wgs84_lng": wgs84_lng,
                "wgs84_lat": wgs84_lat
            }
        ]
    else:
        rep_params1 = {
            "key": amp_api_key,
            "address": amp_api_name,
            #"city": amp_api_city
        }
        response = requests.get(req_url_pref, params=rep_params1)
        data = response.json()
        status = data["status"]
        if status != "0":
                address = data["geocodes"][0]["formatted_address"]
                lng = float(data["geocodes"][0]["location"].split(",")[0])
                lat = float(data["geocodes"][0]["location"].split(",")[1])
                wgs84_lng,wgs84_lat=tbd.gcj02towgs84(lng,lat)
                wgs84_lng=float(wgs84_lng)
                wgs84_lat=float(wgs84_lat)
                busi_data = [
                    {
                        "input_name": amp_api_name,
                        "price":price,
                        "aveprice":aveprice,
                        "input_address": amp_api_address,          
                        "output_address": address,
                        "gcj02_lng": lng,
                        "gcj02_lat": lat,
                        "wgs84_lng": wgs84_lng,
                        "wgs84_lat": wgs84_lat
                        }
                ]
        else:
            busi_data = [
        {
            "input_name": amp_api_name,
            "price":price,
            "aveprice":aveprice,
            "input_address": amp_api_address,
            "output_address": 'Unknow',
            "gcj02_lng": 'Unknow',
            "gcj02_lat": 'Unknow',
            "wgs84_lng": 'Unknow',
            "wgs84_lat": 'Unknow'
        }]
            count += 1
            print(count,":",amp_api_name,"未能在地图中找到位置")
    df = pd.DataFrame(busi_data)
    result = pd.concat([result, df], axis=0, ignore_index=True)   # 将每次i结果union在一块（列方面追加）
 
try:
    # 遍历地址
    for i in range(len(amp_api_data.values)):     
    
        # amp_api_name = amp_api_data.values[i][0]
        # amp_api_address = amp_api_data.values[i][1] 
        amp_api_name = amp_api_data['标题'][i]
        amp_api_address = amp_api_data['listmap'][i]
    
        price=amp_api_data['价格'][i]
        aveprice=amp_api_data['价格10'][i]
        #amp_api_city = '北京市'
        #  请求参数
        rep_params = {
            "key": amp_api_key,
            "address": amp_api_address,
            #"city": amp_api_city
        }
        get_poi_from_amap()
        print("进度：[",i+1,"/",len(amp_api_data),"]", sep='')
        # time.sleep(3)
        sleep(random.randint(1, 3))
except:
    # 将脚本所在路径作为excel输出路径
    # output_path = os.getcwd() + os.sep + "location_gd.xlsx"
    output_path = 'data\HousePrice\上海楼盘location_error.xlsx'
    # 将结果写入到output_path 所在的excel中
    result.to_excel(output_path, index=False)
    print("---------------经纬度获取完成(来源：高德地图)-------------------")
    print("有",count,"个地址未查询到对应的经纬度")
    print("循环到 ",i," 条报错跳出")
        
    
    
 
# 将脚本所在路径作为excel输出路径
# output_path = os.getcwd() + os.sep + "location_gd.xlsx"
output_path = 'data\HousePrice\上海楼盘location1090.xlsx'
# 将结果写入到output_path 所在的excel中
result.to_excel(output_path, index=False)
print("---------------经纬度获取完成(来源：高德地图)-------------------")
print("有",count,"个地址未查询到对应的经纬度")

---------------------------------------------------------------
进度：[1/1130]
进度：[2/1130]
进度：[3/1130]
进度：[4/1130]
进度：[5/1130]
进度：[6/1130]
进度：[7/1130]
进度：[8/1130]
进度：[9/1130]
进度：[10/1130]
进度：[11/1130]
进度：[12/1130]
进度：[13/1130]
进度：[14/1130]
进度：[15/1130]
进度：[16/1130]
进度：[17/1130]
进度：[18/1130]
进度：[19/1130]
进度：[20/1130]
进度：[21/1130]
进度：[22/1130]
进度：[23/1130]
进度：[24/1130]
进度：[25/1130]
进度：[26/1130]
进度：[27/1130]
进度：[28/1130]
进度：[29/1130]
进度：[30/1130]
进度：[31/1130]
进度：[32/1130]
进度：[33/1130]
进度：[34/1130]
进度：[35/1130]
进度：[36/1130]
进度：[37/1130]
进度：[38/1130]
进度：[39/1130]
进度：[40/1130]
进度：[41/1130]
进度：[42/1130]
进度：[43/1130]
进度：[44/1130]
进度：[45/1130]
进度：[46/1130]
进度：[47/1130]
进度：[48/1130]
进度：[49/1130]
进度：[50/1130]
进度：[51/1130]
进度：[52/1130]
进度：[53/1130]
进度：[54/1130]
进度：[55/1130]
进度：[56/1130]
进度：[57/1130]
进度：[58/1130]
进度：[59/1130]
进度：[60/1130]
进度：[61/1130]
进度：[62/1130]
进度：[63/1130]
进度：[64/1130]
进度：[65/1130]
进度：[66/1130]
进度：[67/1130]
进度：[68/1130]
进度：[69/1130]
进度：[70/1130]
进度：[71/1130]
进度：[72/1130]
进度：[73/11

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

data=pd.read_csv('data/HousePrice/上海楼盘location.csv')
data.dropna(inplace=True)

In [5]:
data.to_csv('上海房价location.csv', index=False,encoding='utf-8-sig')